# Tranining a new tokenizer
* A tokenizer will not be suitable if it has been traned on a corpus that is not similar to the one you will use to train your model from scratch
    * Dissimilarites can aries from:-
        * New language
        * New characters
        * New domain
        * New Style  
* you may want to consider training a new tokenizer so that you have a tokenizer suitable for the training used to train a language model from scratch
* To train a new tokenizer these are the necessay steps:-
    * Gather a corpus of text
    * choose tokenizer architecture
    * train the tokenizer on the corpus
    * save the result

In [1]:
from transformers import AutoTokenizer

In [6]:
model_chekpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_chekpoint)

In [7]:
text =" here is a sentence adapted to our tokenizer"
print(tokenizer.tokenize(text))

['here', 'is', 'a', 'sentence', 'adapted', 'to', 'our', 'token', '##izer']


In [9]:
text="उससे धन की सहायता की आशा करत"
print(tokenizer.tokenize(text))

['उ', '##स', '##स', 'ध', '##न', 'क', '##ी', 'स', '##ह', '##ा', '##य', '##त', '##ा', 'क', '##ी', 'आ', '##श', '##ा', 'क', '##र', '##त']


In [10]:
# now we can easily see that this tokenize is not good for hindi words 
#beuase it will seerate or break a simple workd into mant tokens
# so that's why evenry tokenixer is used for different language and for different task
# if we want to use tokenizer then we need to build a new tokenizer which will understand hindi words easily

In [11]:
text = "the medical vocabulary is divided into many sub-toke: paracetamol, phrayngities"
print(tokenizer.tokenize(text))

['the', 'medical', 'vocabulary', 'is', 'divided', 'into', 'many', 'sub', '-', 'to', '##ke', ':', 'para', '##ce', '##tam', '##ol', ',', 'ph', '##ray', '##ng', '##ities']


In [12]:
# There are two ways to build a new tokenizer
#1st by using same arcitecutre of other tokenizer 
#2nd completely desining and building new tokenizer

In [14]:
# The transformer library provides a very easy to use method to train a tokenizer
# using a known architecture on a new corpus
'''AutoTokenizer.train_new_from_iterator(
    text_iterator,
    vocab_size,
    new_special_tokens=None,
    special_tokens_map=None,
    **kwargs
)'''

'AutoTokenizer.train_new_from_iterator(\n    text_iterator,\n    vocab_size,\n    new_special_tokens=None,\n    special_tokens_map=None,\n    **kwargs\n)'

In [15]:
# the first step is to gather a training corpus
from datasets import load_dataset

In [16]:
raw_dataset =  load_dataset("code_search_net","python")

C:\Users\rajkr\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [17]:
# after that
def get_training_corpus():
    dataset = raw_dataset['train']
    for start_idx in range(0,len(dataset),100):
        samples = dataset[start_idx : start_idx+1000]
        yield samples["whole_func_string"]

In [19]:
training_corpus = get_training_corpus()
#loading gpt2 tokenizer because we are going to use it's architecture for building our new tokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\rajkr\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rajkr\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\rajkr\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus,52000)

In [ ]:
# now we can save our new tokenizer
new_tokenizer.save_pretrained("code-search-net-tokenizer")